## This tool is offered by Nao 

![Image](https://static.wixstatic.com/media/88463b_4528450409ad4ff182fd5ac461a98d2f~mv2.png/v1/fill/w_192,h_68,al_c,q_85,usm_0.66_1.00_0.01,enc_auto/88463b_4528450409ad4ff182fd5ac461a98d2f~mv2.png "markdown")

www.nao.school

---

# Stable Diffusion

Stable Diffusion is a text-to-image model. We'll use this model to create our unique Xmas image 🎅

## Setup GPU

First of all, we'll use <b>GPU runtime</b> to run this notebook, so inference is much faster.

⚠️ Use the `Runtime`  menu above and select `Change runtime type`.

## Setup libraries

Then, you just have to run the cell bellow to install some very usefull libraries to use Stable Diffusion

The `%%` characters for Colab built-in magic commands

This `!` character means that we are running shell commands

In [ ]:
# hide the outputs
%%capture

!pip install timm # huggingface_hub 
!pip install transformers scipy ftfy
!pip install diffusers==0.8.0
!pip install stable_diffusion_videos
!pip install "ipywidgets>=7,<8"
!pip install accelerate

## Import

We import some useful libraries with Colab and computer vision

In [ ]:
from diffusers import StableDiffusionPipeline
from stable_diffusion_videos import StableDiffusionWalkPipeline, Interface

from torch import autocast, float16, Generator

from PIL import Image

from IPython.display import HTML
from base64 import b64encode

In [ ]:
# then, we'll load the model Stable Diffusion

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=float16)
pipe = pipe.to("cuda")

`num_inference_steps`: you can change the number of inference steps using the num_inference_steps argument. In general, results are better the more steps you use, however the more steps, the longer the generation takes.



In [ ]:
generator_cat = Generator("cuda").manual_seed(32) # remove manual_seed and the parameter generator to generate a random image

prompt = "a photograph of a very cute cat"
with autocast("cuda"):
  cat_image = pipe(prompt, num_inference_steps=50, generator=generator_cat, height=512, width=512).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

# to save it 
cat_image.save("cat.png")

# or if you're in a google colab you can directly display it with 
cat_image

## Well done 🎉 you have created your first image !

Now it's time to change the sentence of the prompt by your own to create another image 🤓

Some short examples:
- "Rubix Cube in Van Gogh style";
- "Eiffel Tower in kaleidoscope style";
- "A drawing of two dogs";
- "a minimalist pine tree";
- "Feminist in Picasso style"


In [ ]:
prompt = "a Rubix Cube in Pablo Picasso style"
with autocast("cuda"):
  image = pipe(prompt, num_inference_steps=50, height=512, width=512).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

image

## Now we are ready to create lots of images! 

In [ ]:
# this function allow us to display multiple images
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

num_cols = 3 # how many columns of images do you need ?
num_rows = 4 # how many rows of images do you need ?

prompt = ["a pile of gifts" ] * num_cols

all_images = []
for i in range(num_rows):
  with autocast("cuda"):
    images = pipe(prompt, num_inference_steps=50, height=512, width=512).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid